In [415]:
!pip install gensim

In [23]:
import pandas as pd
import numpy as np
import gensim
import re
from tqdm import trange
from sklearn.metrics.pairwise import cosine_similarity
from nltk.corpus import stopwords
import random

In [2]:
df = pd.read_csv("D:\RecSys\products (1).csv")
for i in range(2,33):
    df2 = pd.read_csv("D:\RecSys\products (" + str(i) + ").csv")
    df = pd.concat([df,df2], ignore_index=True)

In [3]:
df.to_csv("D:\RecSys\products.csv")

In [4]:
df[df['article']==628132]

,name,description,article


In [5]:
df.sample(50)

,name,description,article
392,LEGO City Nitro Wheels Гоночные автомобили арт...,LEGO City Nitro Wheels Гоночные автомобили арт...,70385
568,Тилибом Воздушный змей Орел усложненный каркас...,NaN,61262
554,Набор 1TOY Трансботы Инженерный батальон,Набор 1TOY Трансботы Инженерный батальон – отл...,69073
250,Игрушечные стрелы Нерф Элит 12 штук,"Набор из 12 стрел, которые подходят к любому б...",632583
311,Игровой набор TOYS NEO Хозяюшка,NaN,11936
455,Сортер-каталка HAPPY SNAIL,NaN,114503
507,Пазл детский в аммортименте Haio Xiang 500,NaN,121811
602,Модель автомобиля радиоуправляемая джип Taiko ...,NaN,71868
612,Игрушечная машина HOT WHEELS Color Shifters,Машинка HOT WHEELS Color Shifters не даст ваше...,488652
440,LEGO DUPLO Super Heroes Лаборатория супергерое...,NaN,70666


плохие рекомендации если похожих товаров почти нет

In [299]:
class ContentBasedRec():
    def __init__(self):
        self.products = None
        self.interactions = None
        self.get_data() #получение данных в датафрейм

        stop_words = stopwords.words('russian') + ['ассортимент', 'ассортименте', 'ассортим', 'см', 'м', 'шт', 'л', 'г', 'против']
        values = np.zeros(len(stop_words))
        self.stop_words_dict = dict(zip(stop_words, values))
        self.get_products_embeddings()

    def get_data(self):
        df = pd.read_csv("D:\RecSys\products (1).csv")
        for i in range(2,33):
            df2 = pd.read_csv("D:\RecSys\products (" + str(i) + ").csv")
            df = pd.concat([df,df2], ignore_index=True)
        self.products = df.copy()
        
    def get_avg_vector(self, words, model, num_features, index_words_set):
        feature_vec = np.zeros((num_features, ), dtype='float32')
        n_words = 0
        for word in words:
            if word in index_words_set:
                n_words += 1
                feature_vec = np.add(feature_vec, model.wv[word])
        if (n_words > 0):
            feature_vec = feature_vec / n_words
        return feature_vec    
    
    def preprocess(self, text, stop_words_dict):
        word_list = []
        for word in re.findall("[а-яА-Яa-zA-ZёЁ]+", text.lower()):
            if word not in stop_words_dict:
                word_list.append(word)
        return word_list
    
    def get_products_embeddings(self):
        products_descriptions = []
        for i in trange(0,self.products['name'].shape[0]):
            products_descriptions.append(self.preprocess(self.products['name'].loc[i], self.stop_words_dict))
            
        self.model = gensim.models.Word2Vec(
                sentences=products_descriptions,
                vector_size=100,                  #размер эмбеддингов
                window=10,
                min_count=1)
                #iter=100)
        self.index_words_set = set(self.model.wv.index_to_key)
        self.embeddings = np.zeros((len(products_descriptions), 100))
        for i in range(self.embeddings.shape[0]):
            self.embeddings[i] = self.get_avg_vector(products_descriptions[i], self.model, 100, self.index_words_set)
            
    def get_rec_U2I(self, interactions, used):
        interactions = np.unique(interactions)
        user_vector = " ".join(self.products[self.products['article']==_id]['name'].to_string() for _id in interactions)
        user_processed = self.preprocess(user_vector, self.stop_words_dict)
        user_emb = self.get_avg_vector(user_processed, self.model, 100, self.index_words_set)
        metrics = cosine_similarity([user_emb], self.embeddings)
        self.products['metrics']=metrics.reshape(-1)
        print(u"Для пользователя, который взаимодействовал с товарами\n")
        for id_ in interactions:
            print(self.products[self.products['article']==id_]['name'].to_string()+" {}".format(id_))
        print(u"\nТакие рекомендации")
        recomendations = pd.DataFrame(self.products.sort_values(by='metrics', ascending=False).loc[:,['name', 'metrics', 'article']])
        interactions = list(interactions)
        interactions.extend(used)
        for product in interactions:
            recomendations.drop(recomendations[recomendations.article==product].index, inplace = True)
        #print(recomendations[:10])
        return recomendations[:10]

In [300]:
CB = ContentBasedRec()

100%|█████████████████████████████████████████████████████████████████████████████| 645/645 [00:00<00:00, 13723.63it/s]


In [301]:
vec = CB.get_rec_U2I([658631,518796], [])

Для пользователя, который взаимодействовал с товарами

98    CREATIVE Бумага цветная Intensive А4 80г/м2 10... 518796
175    Миксер PHM 7016 электрический Polaris 658631

Такие рекомендации


In [302]:
df.head(50)

,name,description,article
0,Тапочки велюровые METRO PROFESSIONAL с закрыты...,Тапочки домашние H-LINE закрытый носок – обувь...,317774
1,Тапочки ARO одноразовые,"Тапочки ARO одноразовые используются в банях, ...",534408
2,Тапочки велюровые METRO PROFESSIONAL с открыты...,333515,317773
3,Букет №9 Моно Гербера 9,NaN,112604
4,Букет №15 Моно Гербера 15,NaN,112605
5,Летающая тарелка Нордпласт 22 см,Летающая тарелка – популярная игра как для бол...,114425
6,"Скакалка, 2,2 м, 1Toy, цвет в ассортименте",NaN,112669
7,Батут уличный 366х80 см,На данный товар возможен предзаказ!1. Количест...,406502
8,Батут 244,NaN,98339
9,Батут SportElite складной KF-4811,NaN,112098


первый вариант генерации

In [154]:
recsys =  ContentBasedRec()
df_users = pd.DataFrame([],columns = ['uid','item_id', 'name'])
for i in range(0,2):
    items = []
    products = input().split(',')
    products = [int(item) for item in products]
    items.extend(products)
    for j in range(0,4):
        prod_list = random.sample(products,random.randint(1,2))
        results = recsys.get_rec_U2I(prod_list, np.unique(items))['article'].to_list()
        items.extend(results[:random.randint(2,7)])
    names = [df[df['article']==item_id]['name'].to_string() for item_id in items]
    df_users = df_users.append({'uid' : (i+1), 'item_id' : np.unique(items), 'name' : names},ignore_index=True)
    
print(df_users)

100%|█████████████████████████████████████████████████████████████████████████████| 645/645 [00:00<00:00, 14328.53it/s]


74332,98339
Для пользователя, который взаимодействовал с товарами

567    1toy глайдер из пенопласта, сборный, без механ... 74332
8    Батут 244 98339

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

567    1toy глайдер из пенопласта, сборный, без механ... 74332
8    Батут 244 98339

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

8    Батут 244 98339

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

567    1toy глайдер из пенопласта, сборный, без механ... 74332

Такие рекомендации
69819,71868
Для пользователя, который взаимодействовал с товарами

622    Игрушка модель фантазийной машинки 69819
602    Модель автомобиля радиоуправляемая джип Taiko ... 71868

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

622    Игрушка модель фантазийной машинки 69819

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

622    Игрушка модель фантазийной машинки 69819
602    Модел

In [83]:
df_users

,uid,item_id,name
0,1,"[11936, 69819, 71868, 71878, 95461, 95463, 114...","[625 Модель машины WELLY иномарка 1:34, 595..."
1,2,"[47839, 60797, 74332, 98339, 112098, 112099, 1...","[167 Измельчитель GEMLUX GL-MC400, 17 Кн..."


In [87]:
df_users['name'].loc[1]

['167    Измельчитель GEMLUX GL-MC400',
 '17    Книга Кун Мифы Древней Греции ЭКСМО',
 '567    1toy глайдер из пенопласта, сборный, без механ...',
 '8    Батут 244',
 '50    Стакан-непроливайка ARO 2 отделения',
 '27    Карандаши BIC Kids, 24 цвета',
 '1    Тапочки ARO одноразовые',
 '156    Мультипекарь RMB-616/3 REDMOND',
 '190    Блендер погружной GEMLUX GL-SB-809',
 '176    Плита GEMLUX GL-IP28TC',
 '163    Мельница GEMLUX GL-PG-002',
 '190    Блендер погружной GEMLUX GL-SB-809',
 '188    Плита GEMLUX GL-IC28B',
 '176    Плита GEMLUX GL-IP28TC',
 '164    Сушка для фруктов и овощей GEMLUX GL-FD-01R',
 '10    Батут SportElite складной KF-5411',
 '9    Батут SportElite складной KF-4811',
 '563    Дартс на присосках 1Toy',
 '572    Дельтаплан 1TOY в пакете',
 '549    Трансботы Lingvo Zoo 1toy',
 '540    Конструктор Blockformers 1TOY',
 '52    Альбом для рисования акварелью KROYTER 200г/кв...',
 '79    Слизь 1TOY в пакете с вложением-сюрпризом в ас...']

Второй вариант генерации синтетической выборки: генерируем товары, просмотренные пользователями на основе рекомендаций, сделанных рекомендательной системой. Выбираем рандомный товар и просим подобрать систему некоторые рекомендации к нему, затем снова просим подобрать рекомендации для некоторого числа товаров из ранее сгенерированных. Делаем до 7 таких итераций.

In [363]:
recsys =  ContentBasedRec()
df_users = pd.DataFrame([],columns = ['uid','item_id', 'name'])
for i in range(0,200):
    items = []
    product = random.sample(df['article'].to_list(),1)
    items.extend(product)
    prod_list = product
    for j in range(0,7):
        results = recsys.get_rec_U2I(prod_list, np.unique(items))['article'].to_list()
        items.extend(results[random.randint(0,3):random.randint(3,6)]) #сместить в сторону? 2-4
        prod_list = random.sample(items,random.randint(1,len(items))) #увеличить?
    names = [df[df['article']==item_id]['name'].to_string() for item_id in items]
    df_users = df_users.append({'uid' : (i), 'item_id' : np.unique(items), 'name' : names},ignore_index=True)
print(df_users)

100%|█████████████████████████████████████████████████████████████████████████████| 645/645 [00:00<00:00, 12404.29it/s]


Для пользователя, который взаимодействовал с товарами

607    Самолет и Робот 2 в 1 Девилон Военный вертолет 114376

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

606    Набор Devik Toys 2 спортивные машинки 114372
601    Набор Devik Toys 2 спортивные машинки 114375
607    Самолет и Робот 2 в 1 Девилон Военный вертолет 114376
254    Пистолет-робот 2 в 1 Devik Toys Велоцираптор, ... 114378

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

601    Набор Devik Toys 2 спортивные машинки 114375
607    Самолет и Робот 2 в 1 Девилон Военный вертолет 114376
254    Пистолет-робот 2 в 1 Devik Toys Велоцираптор, ... 114378

Такие рекомендации
Для пользователя, который взаимодействовал с товарами

58    Альбом для рисования Bruno Visconti акварелью ... 114325
601    Набор Devik Toys 2 спортивные машинки 114375
607    Самолет и Робот 2 в 1 Девилон Военный вертолет 114376
260    Пистолет робот 2в1 Striker 645297
323    Игровой набор Toys Neo Инстру

In [364]:
df_users.sample(1)['name'].to_list()

[['131    Набор для творчества B&H 19х17 см',
  '135    Набор Рождественский носок B&H',
  '124    Набор Для Творчества Домашняя Кухня Ажурные по...',
  '125    Набор для детского творчества Домики-фонарики',
  '132    Набор новогодняя елка B&H',
  '126    Набор для творчества Ангелочки',
  '133    Набор B AND H Новогодние игрушки',
  '123    Набор для детского творчества Бумажные шары Ка...',
  '130    Набор для детского творчества Нарядные Фанерки',
  '122    Набор для детского творчества Фигурная Гирлянда',
  '35    Набор кистей ГАММА Школьник пони, 3 шт',
  '120    Набор для детского творчества Шарик-фонарик',
  '550    Набор лошадей 3 шт',
  '128    Набор для детского творчества Растущая елка',
  '121    Набор для детского творчества Искрящаяся Снежинка',
  '118    Набор Домашняя кухня для творчества Сосульки и...',
  '127    Набор для творчества Бумажные Шишки',
  '504    Набор карт ИМАДЖИНАРИУМ Пандора',
  '281    Набор Toys Neo Красавица',
  '324    Набор доктора Toys Neo',
  '

длина вектора, min_words, window?

In [365]:
df_users.to_csv("D:\RecSys\syntetic_data.csv")

In [362]:
df[df['article']==70110]

,name,description,article
370,LEGO City Fire Лесные пожарные арт. 60247,NaN,70110
